In [6]:
!unzip drive/My\ Drive/emoji

Archive:  drive/My Drive/emoji.zip
   creating: emoji/
  inflating: emoji/test_emoji.csv    
  inflating: emoji/train_emoji.csv   


In [0]:
import emoji


In [0]:
from tensorboardcolab import *

In [2]:
print(emoji.emojize(":angry_face:"))
print(emoji.emojize(":OK_hand_light_skin_tone:"))

😠
👌🏻


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv("emoji/train_emoji.csv",header=None)

In [5]:
data.head(n=10)

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN
5,I love you mum,0,NaN,NaN
6,Stop saying bullshit,3,NaN,NaN
7,congratulations on your acceptance,2,NaN,NaN
8,The assignment is too long,3,NaN,NaN
9,I want to go play,1,NaN,[3]


In [0]:
emoji_dict = {
    0:":beating_heart:",
    1:":baseball:",
    2:":beaming_face_with_smiling_eyes:",
    3:":disappointed_face:",
    4:":fork_and_knife:"    
}

In [7]:
for ix in emoji_dict.values():
    print(emoji.emojize(ix))

💓
⚾
😁
😞
🍴


In [0]:
X_train = data[0]
Y_train = data[1]

In [9]:
print(X_train.shape)

(132,)


In [10]:
print(Y_train.shape)

(132,)


In [11]:
print(X_train[0])

never talk to me again


In [12]:
print(emoji.emojize(emoji_dict[Y_train[0]]))

😞


In [13]:
#convert every sentence into a list of words
for ix in range(132):
    X_train[ix] = X_train[ix].lower().split()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
print(X_train[1])

['i', 'am', 'proud', 'of', 'your', 'achievements']


In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r',encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map


In [0]:
import numpy as np
words_to_index, index_to_words, word_to_vec_map = read_glove_vecs("glove.6B.50d.txt")

In [24]:
word_to_vec_map["happy"].shape

(50,)

In [0]:
def avg_vector(words):
    
    ans = word_to_vec_map[words[0]]
   # print(ans)
    for i in range(1,len(words)):
        ans = np.add(ans,word_to_vec_map[words[i]])
    ans = ans/float(len(words))
    return ans    

In [0]:
ans = avg_vector(X_train[0])

In [0]:
X_numbers = np.zeros((132,50))

In [0]:
for i in range(X_train.shape[0]):
    X_numbers[i] = avg_vector(X_train[i])

In [29]:
print(X_numbers.shape)
print(X_numbers)

(132, 50)
[[ 0.0971846   0.0174374   0.052843   ... -0.2522034  -0.2197784
   0.307606  ]
 [ 0.00563117  0.59711333 -0.13939717 ... -0.06701233 -0.025773
   0.32180167]
 [ 0.27343125  0.33763875 -0.33053875 ... -0.335001    0.06524875
  -0.04257963]
 ...
 [ 0.29220562  0.1533622  -0.015246   ...  0.032272    0.1313342
   0.472114  ]
 [-0.00701397  0.54196333 -0.19225433 ... -0.80106333 -0.20057967
   0.78087   ]
 [-0.1149985   0.6452325  -0.39727    ...  0.142175   -0.2057708
   0.5231    ]]


In [30]:
from keras.models import Sequential
from keras.layers import Dense,Input,Activation
from keras.utils import to_categorical


Using TensorFlow backend.


In [0]:
model = Sequential()
# Build neural network
model.add(Dense(10, activation='relu', input_shape=(50,)))
model.add(Dense(5, activation='softmax'))

In [0]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [61]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 55        
Total params: 565
Trainable params: 565
Non-trainable params: 0
_________________________________________________________________


In [62]:
print(Y_train.shape)
Y_onehot = to_categorical(Y_train)
print(Y_onehot.shape)

(132,)
(132, 5)


In [0]:
from keras.callbacks import *
cp = ModelCheckpoint('model.h5', monitor = 'val_loss', save_best_only = "True")
es = EarlyStopping(monitor = 'val_acc', patience = 1)

In [64]:
tbc= TensorBoardColab()


Wait for 8 seconds...
TensorBoard link:
https://6d07458a.ngrok.io


In [69]:
model.fit(X_numbers,Y_onehot, validation_split= 0.3, epochs=100, callbacks=[cp,es, TensorBoardColabCallback(tbc)])

Train on 92 samples, validate on 40 samples
Epoch 1/100
92/92 [==============================] - 0s 182us/step - loss: 1.0401 - acc: 0.6848 - val_loss: 1.2491 - val_acc: 0.4750
Epoch 2/100
92/92 [==============================] - 0s 155us/step - loss: 1.0343 - acc: 0.6848 - val_loss: 1.2451 - val_acc: 0.5000
Epoch 3/100
92/92 [==============================] - 0s 159us/step - loss: 1.0277 - acc: 0.6848 - val_loss: 1.2402 - val_acc: 0.5000


In [70]:
i = 7
print(data[0][i])

['congratulations', 'on', 'your', 'acceptance']


In [0]:
output = np.argmax(model.predict(X_numbers),axis=1)

In [72]:
print(Y_train[:10])

0    3
1    2
2    3
3    0
4    4
5    0
6    3
7    2
8    3
9    1
Name: 1, dtype: int64


In [73]:
np.sum(output==Y_train)

83

In [74]:
text = "i love this movie"

avg_v = avg_vector(text.split())
print(avg_v.shape)
avg_v= avg_v.reshape(1,50)
print(avg_v.shape)

(50,)
(1, 50)


In [0]:
emoji_label_predict = np.argmax(model.predict(avg_v))

In [76]:
print(emoji_label_predict)
print(emoji.emojize(emoji_dict[emoji_label_predict]))

2
😁


In [22]:
!ls

drive  glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip  model.h5
emoji  glove.6B.200d.txt  glove.6B.50d.txt   Graph	   sample_data


In [20]:
!unzip glove.6B

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
